## Oefeningen 1: Transacties

In [1]:
from database import Database
db = Database("../oefeningen.db")

In [ ]:
# Selecteer eerste vijf rijen om inzicht te krijgen in de data
db.query("SELECT * FROM Transacties LIMIT 5")

### # Oefening 1.1: Totaal aantal transacties

In [ ]:
db.query("SELECT COUNT(*) FROM Transacties;")

### Oefening 1.2: Hoeveel laag BTW tarief?

In [ ]:
db.query("SELECT COUNT(*) FROM Transacties WHERE BTWTarief = 'laag';")

### Oefening 1.3: Totaal exclusief BTW

In [ ]:
db.query("""
SELECT
    Aantal,
    PrijsExclusief,
    PrijsExclusief * Aantal AS TotaalExclusief,
    BTWTarief
FROM Transacties
;
""")

### Oefening 1.4: Totaal inclusief BTW

In [ ]:
db.query("""
SELECT
    Aantal,
    PrijsExclusief,
    BTWTarief,
    PrijsExclusief * Aantal AS TotaalExclusief,
    CASE
        WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
        ELSE PrijsExclusief * Aantal * 1.21
    END AS TotaalInclusief
FROM Transacties
;
""")

### Oefening 1.5: Grootste / kleinste transacties

In [ ]:
# Grootste transacties, ORDER BY ... DESC
db.query("""
    SELECT
        *,
        CASE
            WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
            ELSE PrijsExclusief * Aantal * 1.21
        END AS TotaalInclusief
    FROM Transacties
    ORDER BY TotaalInclusief DESC
    LIMIT 3
;
""")

In [ ]:
# Kleinste transacties, ORDER BY ... ASC
db.query("""
SELECT *
FROM (
    SELECT
        *,
        CASE
            WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
            ELSE PrijsExclusief * Aantal * 1.21
        END AS TotaalInclusief
    FROM Transacties
)
ORDER BY TotaalInclusief ASC
LIMIT 3
;
""")

### Oefening 1.6: Omzet in Januari

In [ ]:
# Check: Wat is eigenlijk het bereik van de besteldatum?
db.query("SELECT MIN(BestelDatum), MAX(BestelDatum) FROM Transacties")

In [ ]:
# Selecteer alleen bestellingen in januari
db.query("""
SELECT
    COUNT(*) AS AantalTransacties,
    SUM(Aantal) AS AantalItems,
    SUM(TotaalInclusief) AS Omzet
FROM (
    SELECT
        Aantal,
        CASE
            WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
            ELSE PrijsExclusief * Aantal * 1.21
        END AS TotaalInclusief
    FROM Transacties
    WHERE BestelDatum BETWEEN '2020-01-01' AND '2020-01-31'
);
""")

In [ ]:
# Of netter: met een Common Table Expression
db.query("""
WITH TransactiesJanuari AS (
    SELECT
        Aantal,
        CASE
            WHEN BTWTarief = 'laag' THEN PrijsExclusief * Aantal * 1.06
            ELSE PrijsExclusief * Aantal * 1.21
        END AS TotaalInclusief
    FROM Transacties
    WHERE BestelDatum BETWEEN '2020-01-01' AND '2020-01-31'
)
SELECT
    COUNT(*) AS AantalTransacties,
    SUM(Aantal) AS AantalItems,
    SUM(TotaalInclusief) AS Omzet
FROM TransactiesJanuari;
""")